In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from tensorflow.contrib.metrics import f1_score as ms
from PIL import Image
import shutil
import time
import datetime

tf.set_random_seed(1234)

In [2]:
X_train = np.load('X_train_colour_non_6jun_300.npy')
train_mean = np.mean(X_train, axis=0)
train_std = np.std(X_train, axis=0)

In [3]:
regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)

def weight_variable(layer_name,shape):
    return tf.get_variable(layer_name,shape=shape,initializer=tf.contrib.layers.xavier_initializer(seed=1234),regularizer=regularizer)

def bias_variable(layer_name,shape):
    return tf.Variable(tf.zeros(shape=shape),name=layer_name)

def conv_layer(inp, 
               layer_name_bias, 
               layer_name, 
               num_input_channels, 
               filter_size, 
               num_filters, 
               strides,
               ksize_maxpool, 
               kernel_regularizer=regularizer,
               pooling=True, 
               batchnorm=True, 
               dropout=True, 
               keep_prob=0.25):
    
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    weights = weight_variable(layer_name, shape=shape)
    biases = bias_variable(layer_name_bias,shape=[num_filters])
    layer = tf.nn.conv2d(input=inp, filter=weights, strides=[1,strides,strides,1], padding="SAME")
    
    layer= tf.nn.bias_add(layer,biases)
    
    layer = tf.nn.relu(layer)

    if batchnorm:
        layer = tf.layers.batch_normalization(layer)
    
    if pooling:
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1,ksize_maxpool,ksize_maxpool,1],
                               strides=[1,ksize_maxpool,ksize_maxpool,1],
                               padding='SAME')
    if dropout: 
        layer = tf.layers.dropout(layer, keep_prob)
        
    return layer, weights


def flatten_layer(layer):
    num_feat = np.prod(layer.get_shape().as_list()[1:4])
    layer = tf.contrib.layers.flatten(layer)
    
    return layer, num_feat


def fc_layer(input,
             layer_name_bias,
             layer_name,
             num_in,
             num_out,
             relu=True,
             batchnorm=True, 
             dropout=True, 
             keep_prob=0.5):
    
    weights = weight_variable(layer_name,shape=[num_in, num_out])
    biases = bias_variable(layer_name_bias,shape=[num_out])
    
    result = tf.add(tf.matmul(input,weights),biases)
    
    if relu:
        result = tf.nn.relu(result)
    
    if batchnorm:
        result = tf.layers.batch_normalization(result)
        
    if dropout: 
        result = tf.layers.dropout(result, keep_prob)
    
    return result


def softmax_function(input,num_in,num_out,weight_name):
    weights = weight_variable(weight_name,shape=[num_in, num_out])
    biases = bias_variable(shape=[num_out])
    result = tf.matmul(input,weights)+biases
    
    return result, weights


def global_pooling(layer,global_pooling='mean'):
    if global_pooling == 'mean':
        tf.reduce_mean(layer, [1, 2], name='global_pool', keep_dims=True)
    elif global_pooling == 'max':
        tf.reduce_max(layer, [1, 2], name='global_pool', keep_dims=True)

img_h = X_train.shape[1]
img_w = X_train.shape[2]

num_training_data = X_train.shape[0]
num_labels=2

# Number of channels: 1 because greyscale
num_channels = 1

filter_size1 = 5
num_filters1 = 30
strides1 = 2
ksize_maxpool1 = 2

filter_size2 = 3
num_filters2 = 40
strides2 = 1
ksize_maxpool2 = 2

filter_size3 = 3
num_filters3 = 50
strides3 = 1
ksize_maxpool3 = 2

#number of neurons
fc_size=10

#l2 regularization
beta = 0.01

tf.reset_default_graph() 
batch = None

x = tf.placeholder(tf.float32, shape = (batch, img_h, img_w, num_channels),name='x')
y = tf.placeholder(tf.int32, shape = [batch,num_labels], name='y')
keep_prob = tf.placeholder(tf.float32)

conv1, weights1 = conv_layer(x,
                              layer_name='conv1',layer_name_bias='conv1_b',
                              num_input_channels=num_channels,
                              filter_size=filter_size1,
                              num_filters=num_filters1,
                              strides=strides1,
                              ksize_maxpool=ksize_maxpool1,
                              pooling=True,
                              batchnorm=True,
                              dropout=False,
                              keep_prob=keep_prob)

conv2, weights2 = conv_layer(conv1,
                              layer_name='conv2',layer_name_bias='conv2_b',
                              num_input_channels=num_filters1,
                              filter_size=filter_size2,
                              num_filters=num_filters2,
                              strides=strides2,
                              ksize_maxpool=ksize_maxpool2,
                              pooling=True,
                              batchnorm=True,
                              dropout=False,
                              keep_prob=keep_prob)

conv3, weights3 = conv_layer(conv2,
                              layer_name='conv3',layer_name_bias='conv3_b',
                              num_input_channels=num_filters2,
                              filter_size=filter_size3,
                              num_filters=num_filters3,
                              strides=strides3,
                              ksize_maxpool=ksize_maxpool3,
                              pooling=True,
                              batchnorm=True,
                              dropout=True,
                              keep_prob=keep_prob)

conv3 = tf.reduce_mean(conv3, [1, 2], name='global_pool', keep_dims=True)

flattened, num_features = flatten_layer(conv3)

fc1 = fc_layer(flattened,
               layer_name='fc1',layer_name_bias='fc1_b',
               num_in=num_filters3,
               num_out=fc_size,
               relu=True,
               batchnorm=True,
               dropout=True,
               keep_prob=keep_prob)

fc2 = fc_layer(fc1,
               layer_name='fc2',layer_name_bias='fc2_b',
               num_in=fc_size,
               num_out=num_labels,
               relu=False,
               batchnorm=False,
               dropout=True,
               keep_prob=keep_prob)

y_pred = tf.nn.softmax(fc2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.flatten instead.


In [5]:
def resize_and_create_array(filepath):
    
    num_files = sum([len(files) for r, d, files in os.walk(filepath)])
    root = [root for root, folder, files in os.walk(filepath)][0]
    X_pred = np.zeros(shape=(num_files, img_h, img_w), dtype='uint8')
    
    i=0
    for root,folder,files in os.walk(filepath):
        for file in files[:]:
            file=file.lower()
            path = os.path.join(root,file)
            if '.png' not in path:
                continue
            try:
                img = Image.open(path).convert('L')
                img = np.array(img.resize((int(img_w),int(img_h))))
                X_pred[i]=img
                i+=1
            except:
                print(file)
    
    X_pred= np.reshape(X_pred,(-1, img_h, img_w,1))
    X_pred = np.divide((X_pred - train_mean), train_std)
    return X_pred
                
    
def create_minibatch_pred(X,batch_size=32):
    
    num_training_data = X.shape[0]
    num_of_batches = num_training_data // batch_size
    minibatch = []
    
    for batch in range(num_of_batches):
        batch_data = X[batch*batch_size:(batch+1)*batch_size, :, :, :]
        minibatch.append(batch_data)
    
    if num_training_data % batch_size != 0:
        batch_data = X[num_of_batches*batch_size:num_training_data, :, :, :]
        minibatch.append(batch_data)
        
    return minibatch


def predict_batch_2(X,batch_size=32):
    output=[]
    accs=0
    saver = tf.train.Saver()
    with tf.Session() as session:
        print(save_path)
        saver.restore(session, save_path)
        minibatches = create_minibatch_pred(X,batch_size)
        for minibatch in minibatches:
            batch_data = minibatch    
            feed_dict = {x: batch_data, keep_prob: 1.0}
            pred = session.run(y_pred, feed_dict=feed_dict)
            pred = np.argmax(np.array(pred, copy=True),1)         
            output.append(pred)
            
        output = np.array(np.concatenate(output).ravel().tolist())
        return output
    
    
def binning_pred(output):
    files = [files for root, folder, files in os.walk(filepath)][0]
    root = [root for root, folder, files in os.walk(filepath)][0]
    
    num_class = len(np.unique(output))
    
    i=0
    for value in output:
        path = os.path.join(root,files[i])
        out_folder = 'class_'+str(value)
        outdir = os.path.join(root,out_folder)
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        shutil.copy(path,outdir)
        i+=1
            
def main_predict():
    start_time = time.time()
    X_pred = resize_and_create_array(filepath)
    print('Pred set', X_pred.shape)
    output = predict_batch_2(X_pred)
    binning_pred(output)
    time_diff = time.time() - start_time
    print('Prediction Done! Elapsed Time: %.2f s' %time_diff)

save_dir = r'C:\All Code\Work\CNN\Coloured_Tables_Detection\coloured_model'
save_path = os.path.join(save_dir, 'coloured_detection')

filepath = input("Prediction Data Path: ")
main_predict()

Prediction Data Path: C:\All Code\Work\CNN\Prediction Data
Pred set (1364, 300, 300, 1)
C:\All Code\Work\CNN\Coloured_Tables_Detection\coloured_model\coloured_detection
INFO:tensorflow:Restoring parameters from C:\All Code\Work\CNN\Coloured_Tables_Detection\coloured_model\coloured_detection
Prediction Done! Elapsed Time: 20.69 s
